# WinDbg notebooks

This is a demo of putting windbg into Polyglot Notebooks.

## Setup

This is boilerplate code.  The `#!import` will not be needed in the future, we'd just reference the WinDbgKernel nuget package with `#r`.

In [1]:
#r "nuget:Microsoft.Diagnostics.Runtime"
#r ".\WinDbgKernel\bin\Debug\net8.0-windows10.0.17763.0\publish\WinDbgKernel.dll"
#!import ".\WinDbgKernel\extension.dib"

// System.Diagnostics.Debugger.Launch();

Installed Packages Microsoft.Diagnostics.Runtime, 3.1.512801

In [2]:
using Microsoft.Diagnostics.Runtime;

DataTarget dt = DataTarget.LoadDump(@"d:\work\crash.dmp");
ClrRuntime runtime = dt.ClrVersions[0].CreateRuntime();

# Load the dump in windbg

This is how we would initialize windbg and point it to a dumpfile.

I have not implemented syntax highlighting and intellisense yet.  Even without it, note that GitHub copilot works here for WinDbg commands without us having to do anything!

In [3]:
#!verboseSymbols false
#!dbgengPath d:\amd64
#!sympath SRV*d:\symbols*http://symweb.corp.microsoft.com
#!loadDump "d:\work\crash.dmp"

* Comments in WinDbg start with an asterisk
* This is boilerplate to load a better version of sos.  In the future this will be taken care of for us.

.unload
.load C:\Users\leculver\.dotnet\sos\sos.dll

* Show the last event and frame where we are stopped at.

.lastevent
.frame

Dump file 'd:\work\crash.dmp' loaded successfully.
0:000> .unload
Unloading d:\my\sym\SO\SOS_AMD64_AMD64_4.8.9181.00.dll\64B854789a4000\SOS_AMD64_AMD64_4.8.9181.00.dll extension DLL
0:000> .load C:\Users\leculver\.dotnet\sos\sos.dll
0:000> .lastevent
Last event: 7298.5178: Break instruction exception - code 80000003 (first/second chance not available)
  debugger time: Fri Jun 28 15:25:41.035 2024 (UTC - 7:00)
0:000> .frame
00 000000f1`e2afebc8 00007ffa`5eef183e     win32u!ZwUserMsgWaitForMultipleObjectsEx+0x14 [onecoreuap\windows\core\umode\moderncore\objfre\amd64\usrstubs.asm @ 9901]


In [4]:
#!share --from windbg curprocess --as curprocess

Console.WriteLine(curprocess.Id);
Console.WriteLine(curprocess.Name);

ClrThread finalizerThread = runtime.Threads.Single(t => t.IsFinalizer);
Console.WriteLine(finalizerThread.OSThreadId);

foreach (var frame in await curprocess.Threads.Single(x => x.Id == finalizerThread.OSThreadId).GetStackTraceAsync())
{
    Console.WriteLine($"{frame.Display}");
    foreach (var local in frame.Locals)
    {
        Console.WriteLine($"  {local.Type} {local.Name} = {local.Value}");
    }
    foreach (var param in frame.Parameters)
    {
        Console.WriteLine($"  {param.Type} {param.Name} = {param.Value}");
    }
}

Error: (1,30): error CS1061: 'JsonDocument' does not contain a definition for 'Id' and no accessible extension method 'Id' accepting a first argument of type 'JsonDocument' could be found (are you missing a using directive or an assembly reference?)
(2,30): error CS1061: 'JsonDocument' does not contain a definition for 'Name' and no accessible extension method 'Name' accepting a first argument of type 'JsonDocument' could be found (are you missing a using directive or an assembly reference?)
(7,40): error CS1061: 'JsonDocument' does not contain a definition for 'Threads' and no accessible extension method 'Threads' accepting a first argument of type 'JsonDocument' could be found (are you missing a using directive or an assembly reference?)
(9,26): error CS1503: Argument 1: cannot convert from 'method group' to 'scoped System.ReadOnlySpan<char>'

# Using SOS

Now that we have SOS and ClrMD loaded, we can run any debugger commands.  For example, let's take a look at what our virtual address space looks like with the managed aware `!maddress`:

In [4]:
!maddress -stat

!dumpheap -type System.Net.HttpWebRequest

0:000> !maddress -stat
 +------------------------------------------------------------------------------------+ 
 | Memory Type                       |          Count |         Size |   Size (bytes) | 
 +------------------------------------------------------------------------------------+ 
 | Image                             |         16,364 |       1.69gb |  1,813,696,512 | 
 | GCHeap                            |              4 |     349.20mb |    366,166,016 | 
 | MAPPED_READONLY                   |            256 |      89.32mb |     93,663,232 | 
 | Heap                              |            106 |      74.98mb |     78,626,816 | 
 | Other                             |            435 |      72.27mb |     75,780,096 | 
 | PAGE_READWRITE                    |            111 |      69.50mb |     72,876,032 | 
 | PAGE_READWRITE, PAGE_WRITECOMBINE |             14 |      28.60mb |     29,990,912 | 
 | HighFrequencyHeap                 |            182 |      11.76mb |     12,333,056 |

Enumerating and tagging the entire address space and caching the result...
Subsequent runs of this command should be faster.


# DbgEng object model

DbgEng has an object model, which is accessed through the `dx` command.

The important thing to realize is that we can project this object model back into C# code.

In [19]:
dx @$curprocess.Threads[0x5178].Stack.Frames


0:000> dx @$curprocess.Threads[0x5178].Stack.Frames
@$curprocess.Threads[0x5178].Stack.Frames
    [0x0]            : win32u!ZwUserMsgWaitForMultipleObjectsEx + 0x14
    [0x1]            : VsLog!VSResponsiveness::Detours::DetourMsgWaitForMultipleObjectsEx + 0x6e
    [0x2]            : msenv!MainMessageLoop::BlockingWait + 0x27
    [0x3]            : msenv!CMsoCMHandler::EnvironmentMsgLoop + 0x209
    [0x4]            : msenv!CMsoCMHandler::FPushMessageLoop + 0x73
    [0x5]            : msenv!SCM::FPushMessageLoop + 0x105
    [0x6]            : msenv!SCM_MsoCompMgr::FPushMessageLoop + 0x3f
    [0x7]            : msenv!CMsoComponent::PushMsgLoop + 0x3d
    [0x8]            : msenv!VStudioMainLogged + 0x881
    [0x9]            : msenv!VStudioMain + 0xdc
    [0xa]            : devenv!util_CallVsMain + 0x5c
    [0xb]            : devenv!CDevEnvAppId::Run + 0x1e03
    [0xc]            : devenv!WinMain + 0xd0
    [0xd]            : devenv!invoke_main + 0x21
    [0xe]            : devenv!__scr

# Using @$currprocess in C#

Not yet implemented, but projecting these values back into C# is straight forward to implement.  To use it, simply use the variable name as you would expect.  Enumerating `@$curprocess.Threads` would look something like this:

In [5]:
foreach ((uint osid, Thread thread) in currprocess.Threads)
{
   // Do something
}

bool isFinalizerWaiting = false;
uint finalizerThread = runtime.Threads.FirstOrDefault(t => t.IsFinalizer)?.OSThreadId ?? 0;
foreach (var stackFrame in currprocess.Threads[finalizerThread].Stack.Frames)
{
    if (stackFrame == "coreclr!WaitForFinalizerEvent")
    {
        isFinalizerWaiting = true;
        break;
    }
}

Error: (1,40): error CS0103: The name 'currprocess' does not exist in the current context
(1,22): error CS0246: The type or namespace name 'Thread' could not be found (are you missing a using directive or an assembly reference?)
(8,28): error CS0103: The name 'currprocess' does not exist in the current context

# Using C# variables in WinDbg

Though the uses will be more limited, we can also project C# variables back into WinDbg.  I've arbitrarily chosen `$` to distinguish between real windbg pseudo variables `@$` but this is modifyable.

In [ ]:
* Swap to the finalizer thread with the '~~[osid]s' syntax

~~[$finalizerThread]s
kn


# Building interactive code

Note that notebooks aren't fully static either.  You can write code which adds code to cells.  While the example below is contrived, imagine the start of a notebook essentially points you at next steps and generates WinDbg commands that would be useful to run.

In [20]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;

var command = new SendEditableCode(
    "windbg", 
    "kn\n!clrthreads");

var input = await Kernel.Root.SendAsync(command);


In [ ]:
kn
!clrthreads